In [8]:
#Just some imports you will need
import mlflow
import jwt
import json
import warnings
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import logging
import os
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [9]:
print('MLFLOW_TRACKING_URI: ' + os.environ['MLFLOW_TRACKING_URI'])

MLFLOW_TRACKING_URI: http://127.0.0.1:8765


In [10]:
client = mlflow.tracking.MlflowClient()

In [11]:
myprefix=''

from datetime import datetime
now = datetime.now()
date_time_str = now.strftime("%m-%d-%Y")

experiment_name = 'demo'+'-' + os.environ['DOMINO_STARTING_USERNAME'] + '-' + os.environ['DOMINO_PROJECT_NAME']
model_name = 'demo'+'-' + os.environ['DOMINO_PROJECT_NAME']
if myprefix!='':
    experiment_name = myprefix + '-' + experiment_name
print(experiment_name)

demo-dev_uk1-API_Deployment_Demo


In [13]:
experiment = client.get_experiment_by_name(name=experiment_name)
#if(experiment is None):
#    print('Creating experiment ')
#    client.create_experiment(name=experiment_name)
#    experiment = client.get_experiment_by_name(name=experiment_name)

print(experiment_name)
mlflow.set_experiment(experiment_name=experiment_name)

MlflowException: API request to endpoint /api/2.0/mlflow/experiments/get-by-name failed with error code 401 != 200. Response body: ''

In [6]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [7]:
# Read the wine-quality csv file from the URL
csv_url = (
     "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
)
try:
     data = pd.read_csv(csv_url, sep=";")
except Exception as e:
     logger.exception(
          "Unable to download training & test CSV, check your internet connection. Error: %s", e
      )

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]


my_log = "This is a test log"
with open("/tmp/test.txt", 'w') as f:
    f.write(my_log)
with open("/tmp/test.log", 'w') as f:
    f.write(my_log)

    
    
#run_tags={'mlflow.user':os.environ['DOMINO_STARTING_USERNAME']}
#Change user name
alpha = 0.7
l1_ratio = 0.6
while(alpha<1):
    with mlflow.start_run():
        print('Start Run')
        print('Alpha : ' + str(alpha))
        print('L1_Ratio : ' + str(l1_ratio))
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        print(mlflow.get_tracking_uri())
        print(tracking_url_type_store)
        # Model registry does not work with file store
        if tracking_url_type_store != "file":
            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            #mlflow.sklearn.log_model(lr, os.environ['DOMINO_STARTING_USERNAME']+"/model", registered_model_name="DEMO-ElasticnetWineModel-11")
            mlflow.sklearn.log_model(lr, "demo_model", registered_model_name=model_name)

        else:        
            mlflow.sklearn.log_model(lr,  model_name)
        print(mlflow.get_artifact_uri())
        alpha=alpha+0.1
        l1_ratio = l1_ratio + 0.05        
        

Start Run
Alpha : 0.7
L1_Ratio : 0.6
Elasticnet model (alpha=0.700000, l1_ratio=0.600000):
  RMSE: 0.8081161187128952
  MAE: 0.6724711167246201
  R2: 0.035294053126874725
http://127.0.0.1:8765
http


Successfully registered model 'demo-API_Deployment_Demo'.
2024/05/13 15:54:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: demo-API_Deployment_Demo, version 1
Created version '1' of model 'demo-API_Deployment_Demo'.


mlflow-artifacts:/mlflow/4f1e10bda459479d907c3fe594ff5af9/artifacts
Start Run
Alpha : 0.7999999999999999
L1_Ratio : 0.65
Elasticnet model (alpha=0.800000, l1_ratio=0.650000):
  RMSE: 0.8088760689681395
  MAE: 0.6739857952388139
  R2: 0.033478786163029906
http://127.0.0.1:8765
http


/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'demo-API_Deployment_Demo' already exists. Creating a new version of this model...
2024/05/13 15:54:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: demo-API_Deployment_Demo, version 2
Created version '2' of model 'demo-API_Deployment_Demo'.


mlflow-artifacts:/mlflow/b87984388a9d476bb110522d925b8080/artifacts
Start Run
Alpha : 0.8999999999999999
L1_Ratio : 0.7000000000000001
Elasticnet model (alpha=0.900000, l1_ratio=0.700000):
  RMSE: 0.8096674664275043
  MAE: 0.6752929041060317
  R2: 0.03158658870676645
http://127.0.0.1:8765
http


/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'demo-API_Deployment_Demo' already exists. Creating a new version of this model...
2024/05/13 15:54:07 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: demo-API_Deployment_Demo, version 3
Created version '3' of model 'demo-API_Deployment_Demo'.


mlflow-artifacts:/mlflow/4bd5a3449f25443ab2a5a1dc08bc3f59/artifacts
Start Run
Alpha : 0.9999999999999999
L1_Ratio : 0.7500000000000001
Elasticnet model (alpha=1.000000, l1_ratio=0.750000):
  RMSE: 0.810047320467269
  MAE: 0.6761290910948322
  R2: 0.030677716728749638
http://127.0.0.1:8765
http


/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'demo-API_Deployment_Demo' already exists. Creating a new version of this model...
2024/05/13 15:54:10 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: demo-API_Deployment_Demo, version 4
Created version '4' of model 'demo-API_Deployment_Demo'.


mlflow-artifacts:/mlflow/33a83b7544e24b72a2056dc0a9cca625/artifacts
